CONVERSATIONAL AI AGENT TEMPLATE (LANGCHAIN)
============================================
Use Case: Task-Oriented Chatbot, Personal Assistant, Customer Support Bot

# 1. PROJECT SETUP & ENVIRONMENT

## 1.1 Install Required Libraries

In [ ]:
# !pip install langchain langchain-community langchain-openai
# !pip install openai anthropic
# !pip install langchain-experimental
# !pip install duckduckgo-search wikipedia
# !pip install gradio streamlit
# !pip install pandas matplotlib requests beautifulsoup4

## 1.2 Import Libraries

In [ ]:
import os
import json
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# LangChain Core
from langchain.agents import (
    Tool,
    AgentExecutor,
    create_react_agent,
    create_structured_chat_agent,
    initialize_agent,
    AgentType
)
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.memory import (
    ConversationBufferMemory,
    ConversationSummaryMemory,
    ConversationBufferWindowMemory,
    ConversationEntityMemory
)

# LLMs
from langchain.chat_models import ChatOpenAI, ChatAnthropic
from langchain.llms import OpenAI, HuggingFacePipeline

# Tools
from langchain.tools import (
    DuckDuckGoSearchRun,
    WikipediaQueryRun,
    YouTubeSearchTool,
    PythonREPLTool
)
from langchain.utilities import (
    WikipediaAPIWrapper,
    SerpAPIWrapper,
    OpenWeatherMapAPIWrapper
)

# Chains
from langchain.chains import (
    LLMChain,
    ConversationChain,
    SimpleSequentialChain,
    SequentialChain,
    LLMMathChain
)

# Callbacks
from langchain.callbacks import StreamingStdOutCallbackHandler

In [ ]:
# Set API keys
os.environ['OPENAI_API_KEY'] = 'your-api-key'
os.environ['SERPAPI_API_KEY'] = 'your-serpapi-key'
os.environ['OPENWEATHERMAP_API_KEY'] = 'your-weather-key'

## 1.3 Configuration

In [ ]:
CONFIG = {
    # LLM configuration
    'llm_provider': 'openai',  # 'openai', 'anthropic', 'huggingface'
    'model_name': 'gpt-3.5-turbo',
    'temperature': 0.7,
    'max_tokens': 1000,
    
    # Agent configuration
    'agent_type': 'structured-chat',  # 'zero-shot-react', 'structured-chat', 'conversational-react'
    'max_iterations': 10,
    'verbose': True,
    
    # Memory configuration
    'memory_type': 'buffer',  # 'buffer', 'summary', 'window', 'entity'
    'memory_window': 5,
    
    # Tool configuration
    'enable_search': True,
    'enable_wikipedia': True,
    'enable_calculator': True,
    'enable_python': False,  # Security risk if enabled
    
    'random_seed': 42
}

# 2. LLM INITIALIZATION

## 2.1 Initialize Language Model

In [ ]:
if CONFIG['llm_provider'] == 'openai':
    llm = ChatOpenAI(
        model_name=CONFIG['model_name'],
        temperature=CONFIG['temperature'],
        max_tokens=CONFIG['max_tokens'],
        streaming=True,
        callbacks=[StreamingStdOutCallbackHandler()]
    )
elif CONFIG['llm_provider'] == 'anthropic':
    llm = ChatAnthropic(
        model=CONFIG['model_name'],
        temperature=CONFIG['temperature'],
        max_tokens=CONFIG['max_tokens']
    )
else:
    # HuggingFace local model
    from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
    
    tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
    model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
    
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=CONFIG['max_tokens']
    )
    
    llm = HuggingFacePipeline(pipeline=pipe)

print(f"LLM initialized: {CONFIG['llm_provider']} - {CONFIG['model_name']}")

# 3. CUSTOM TOOLS CREATION

## 3.1 Web Search Tool

In [ ]:
if CONFIG['enable_search']:
    search = DuckDuckGoSearchRun()
    
    search_tool = Tool(
        name="Web Search",
        func=search.run,
        description="Useful for searching the internet for current information. Input should be a search query."
    )
else:
    search_tool = None

## 3.2 Wikipedia Tool

In [ ]:
if CONFIG['enable_wikipedia']:
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    
    wiki_tool = Tool(
        name="Wikipedia",
        func=wikipedia.run,
        description="Useful for getting detailed information about topics from Wikipedia. Input should be a topic or entity name."
    )
else:
    wiki_tool = None

## 3.3 Calculator Tool

In [ ]:
if CONFIG['enable_calculator']:
    from langchain_experimental.llm_math.base import LLMMathChain
    
    math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
    
    calculator_tool = Tool(
        name="Calculator",
        func=math_chain.run,
        description="Useful for performing mathematical calculations. Input should be a mathematical expression."
    )
else:
    calculator_tool = None

## 3.4 Custom API Tools

In [ ]:
import requests

def get_weather(location: str) -> str:
    """Get current weather for a location"""
    try:
        # Use OpenWeatherMap or similar API
        api_key = os.environ.get('OPENWEATHERMAP_API_KEY', '')
        if not api_key:
            return "Weather API key not configured"
        
        url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=metric"
        response = requests.get(url)
        data = response.json()
        
        if response.status_code == 200:
            temp = data['main']['temp']
            description = data['weather'][0]['description']
            return f"Weather in {location}: {temp}°C, {description}"
        else:
            return f"Could not fetch weather for {location}"
    except Exception as e:
        return f"Error fetching weather: {str(e)}"

weather_tool = Tool(
    name="Weather",
    func=get_weather,
    description="Get current weather information for a location. Input should be a city name."
)

In [ ]:
def get_current_time() -> str:
    """Get current date and time"""
    now = datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S")

time_tool = Tool(
    name="Current Time",
    func=lambda x: get_current_time(),
    description="Get the current date and time. No input needed."
)

In [ ]:
def create_reminder(reminder_text: str) -> str:
    """Create a reminder (simulated)"""
    # In production, this would save to a database
    with open('reminders.txt', 'a') as f:
        f.write(f"{datetime.now()}: {reminder_text}\n")
    return f"Reminder created: {reminder_text}"

reminder_tool = Tool(
    name="Create Reminder",
    func=create_reminder,
    description="Create a reminder for later. Input should be the reminder text."
)

## 3.5 Database Query Tool

In [ ]:
def query_database(query: str) -> str:
    """Query a database (simulated)"""
    # Simulated database
    database = {
        "user_count": 1000,
        "active_users": 750,
        "revenue": "$50,000"
    }
    
    query_lower = query.lower()
    
    if "user" in query_lower and "count" in query_lower:
        return f"Total users: {database['user_count']}"
    elif "active" in query_lower:
        return f"Active users: {database['active_users']}"
    elif "revenue" in query_lower:
        return f"Revenue: {database['revenue']}"
    else:
        return "Available metrics: user_count, active_users, revenue"

database_tool = Tool(
    name="Database Query",
    func=query_database,
    description="Query the database for metrics and statistics. Input should be a query about users, revenue, or activity."
)

## 3.6 Assemble Tools List

In [ ]:
tools = []

if search_tool:
    tools.append(search_tool)
if wiki_tool:
    tools.append(wiki_tool)
if calculator_tool:
    tools.append(calculator_tool)

tools.extend([weather_tool, time_tool, reminder_tool, database_tool])

print(f"\nAvailable tools ({len(tools)}):")
for tool in tools:
    print(f"  - {tool.name}: {tool.description}")

# 4. MEMORY CONFIGURATION

## 4.1 Initialize Memory

In [ ]:
if CONFIG['memory_type'] == 'buffer':
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True
    )
elif CONFIG['memory_type'] == 'summary':
    memory = ConversationSummaryMemory(
        llm=llm,
        memory_key="chat_history",
        return_messages=True
    )
elif CONFIG['memory_type'] == 'window':
    memory = ConversationBufferWindowMemory(
        k=CONFIG['memory_window'],
        memory_key="chat_history",
        return_messages=True
    )
elif CONFIG['memory_type'] == 'entity':
    memory = ConversationEntityMemory(
        llm=llm,
        memory_key="chat_history",
        return_messages=True
    )

print(f"Memory initialized: {CONFIG['memory_type']}")

# 5. AGENT CREATION

## 5.1 Create System Prompt

In [ ]:
system_prompt = """You are a helpful AI assistant with access to various tools.

Your goal is to help users with their requests by:
1. Understanding what they need
2. Using the appropriate tools when necessary
3. Providing clear and helpful responses
4. Asking for clarification if needed

When using tools:
- Choose the most appropriate tool for the task
- Provide clear inputs to tools
- Explain your reasoning

Always be polite, professional, and helpful."""

## 5.2 Initialize Agent

In [ ]:
# Create agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=CONFIG['verbose'],
    max_iterations=CONFIG['max_iterations'],
    handle_parsing_errors=True
)

# Update system message
agent.agent.llm_chain.prompt.messages[0] = SystemMessagePromptTemplate.from_template(
    system_prompt
)

print("Agent initialized successfully")

# 6. CONVERSATION INTERFACE

## 6.1 Basic Chat Function

In [ ]:
def chat(message, agent, memory):
    """Send a message to the agent"""
    try:
        response = agent.run(message)
        return response
    except Exception as e:
        return f"Error: {str(e)}"

## 6.2 Interactive Chat Loop

In [ ]:
def interactive_chat(agent):
    """Interactive chat loop"""
    print("\n" + "="*60)
    print("AI Assistant - Type 'quit' to exit")
    print("="*60 + "\n")
    
    while True:
        user_input = input("You: ")
        
        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("Assistant: Goodbye! Have a great day!")
            break
        
        if not user_input.strip():
            continue
        
        print("\nAssistant: ", end="")
        response = chat(user_input, agent, memory)
        print(response + "\n")

# Run interactive chat
# interactive_chat(agent)

## 6.3 Test Conversations

In [ ]:
test_conversations = [
    "What's the weather like in London?",
    "Can you search for the latest news about artificial intelligence?",
    "What is 25 * 17 + 103?",
    "Tell me about Albert Einstein",
    "What time is it now?",
    "Create a reminder to buy groceries tomorrow"
]

print("\n" + "="*60)
print("TEST CONVERSATIONS")
print("="*60)

for message in test_conversations:
    print(f"\nUser: {message}")
    print("Assistant: ", end="")
    response = chat(message, agent, memory)
    print(response)
    print("-" * 60)

# 7. MULTI-TURN CONVERSATIONS

## 7.1 Context-Aware Conversations

In [ ]:
multi_turn_conversation = [
    "I'm planning a trip to Paris",
    "What's the weather there?",
    "Can you search for popular tourist attractions?",
    "How about the Louvre Museum?",
    "Create a reminder to book tickets"
]

print("\n" + "="*60)
print("MULTI-TURN CONVERSATION")
print("="*60)

# Reset memory for clean start
memory.clear()

for message in multi_turn_conversation:
    print(f"\nUser: {message}")
    print("Assistant: ", end="")
    response = chat(message, agent, memory)
    print(response)
    print()

## 7.2 View Conversation History

In [ ]:
print("\n" + "="*60)
print("CONVERSATION HISTORY")
print("="*60)
print(memory.buffer)

# 8. SPECIALIZED AGENTS

## 8.1 Customer Support Agent

In [ ]:
customer_support_prompt = """You are a customer support AI assistant for an e-commerce company.

Your responsibilities:
- Help customers with order inquiries
- Provide product information
- Handle returns and refunds
- Address complaints professionally
- Escalate complex issues to human agents

Always:
- Be empathetic and understanding
- Provide clear solutions
- Ask for order numbers when relevant
- Follow company policies

Available actions:
- Check order status
- Process returns
- Look up product details
- Create support tickets"""

def create_customer_support_agent(llm, tools):
    """Create specialized customer support agent"""
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True
    )
    
    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
        memory=memory,
        verbose=True
    )
    
    return agent

## 8.2 Personal Assistant Agent

In [ ]:
personal_assistant_prompt = """You are a personal AI assistant helping with daily tasks.

Capabilities:
- Schedule management
- Reminders and todos
- Information lookup
- Email and message drafting
- Travel planning
- General questions

Personality:
- Friendly and casual
- Proactive in suggestions
- Organized and efficient
- Respectful of privacy"""

## 8.3 Data Analyst Agent

In [ ]:
def create_data_analyst_agent(llm):
    """Create agent specialized in data analysis"""
    
    # Add data analysis tools
    def analyze_data(query: str) -> str:
        """Analyze data based on query"""
        # Simulated analysis
        return f"Analysis result for: {query}"
    
    data_tool = Tool(
        name="Data Analysis",
        func=analyze_data,
        description="Analyze data and generate insights"
    )
    
    analysis_tools = [data_tool, calculator_tool, database_tool]
    
    agent = initialize_agent(
        tools=analysis_tools,
        llm=llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )
    
    return agent

# 9. ADVANCED FEATURES

## 9.1 Tool Selection Strategy

In [ ]:
from langchain.agents.agent_toolkits import create_retriever_tool

def smart_tool_selection(query: str, tools: list) -> list:
    """Select most relevant tools for a query"""
    # Implement embedding-based tool selection
    relevant_tools = []
    
    query_lower = query.lower()
    
    # Simple keyword matching (can be improved with embeddings)
    if any(word in query_lower for word in ['weather', 'temperature', 'climate']):
        relevant_tools.append([t for t in tools if t.name == 'Weather'][0])
    
    if any(word in query_lower for word in ['search', 'find', 'look up']):
        relevant_tools.append([t for t in tools if t.name == 'Web Search'][0])
    
    if any(word in query_lower for word in ['calculate', 'math', 'compute']):
        relevant_tools.append([t for t in tools if t.name == 'Calculator'][0])
    
    # Return all tools if no specific match
    return relevant_tools if relevant_tools else tools

## 9.2 Intent Classification

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

intent_template = """Classify the user's intent into one of these categories:
- question: User is asking for information
- task: User wants to perform an action
- conversation: Casual conversation
- complaint: User has an issue

User message: {message}

Intent:"""

intent_prompt = PromptTemplate(template=intent_template, input_variables=["message"])
intent_chain = LLMChain(llm=llm, prompt=intent_prompt)

def classify_intent(message: str) -> str:
    """Classify user intent"""
    intent = intent_chain.run(message=message).strip().lower()
    return intent

# Test intent classification
test_messages = [
    "What's the capital of France?",
    "Create a reminder for my meeting",
    "How are you doing today?",
    "This product is broken!"
]

print("\n" + "="*60)
print("INTENT CLASSIFICATION")
print("="*60)

for msg in test_messages:
    intent = classify_intent(msg)
    print(f"Message: {msg}")
    print(f"Intent: {intent}\n")

## 9.3 Sentiment Analysis

In [ ]:
sentiment_template = """Analyze the sentiment of this message:
"{message}"

Sentiment (positive/negative/neutral):"""

sentiment_prompt = PromptTemplate(template=sentiment_template, input_variables=["message"])
sentiment_chain = LLMChain(llm=llm, prompt=sentiment_prompt)

def analyze_sentiment(message: str) -> str:
    """Analyze message sentiment"""
    sentiment = sentiment_chain.run(message=message).strip().lower()
    return sentiment

## 9.4 Response Quality Evaluation

In [ ]:
def evaluate_response(user_message: str, agent_response: str) -> dict:
    """Evaluate the quality of agent response"""
    metrics = {
        'length': len(agent_response),
        'has_greeting': any(word in agent_response.lower() for word in ['hello', 'hi', 'hey']),
        'is_polite': any(word in agent_response.lower() for word in ['please', 'thank', 'sorry']),
        'sentiment': analyze_sentiment(agent_response)
    }
    
    return metrics

# 10. GRADIO INTERFACE

## 10.1 Create Chatbot Interface

In [ ]:
import gradio as gr

def create_chatbot_interface(agent):
    """Create Gradio chatbot interface"""
    
    def respond(message, chat_history):
        # Get response from agent
        bot_response = chat(message, agent, memory)
        
        # Update history
        chat_history.append((message, bot_response))
        
        return "", chat_history
    
    with gr.Blocks(title="AI Assistant") as demo:
        gr.Markdown("# 🤖 AI Assistant")
        gr.Markdown("Ask me anything! I can search the web, do calculations, and more.")
        
        chatbot = gr.Chatbot(height=500)
        msg = gr.Textbox(
            placeholder="Type your message here...",
            label="Message"
        )
        
        with gr.Row():
            submit = gr.Button("Send")
            clear = gr.Button("Clear")
        
        # Examples
        gr.Examples(
            examples=[
                "What's the weather in Tokyo?",
                "Calculate 15% of 250",
                "Search for latest AI news",
                "What time is it?"
            ],
            inputs=msg
        )
        
        msg.submit(respond, [msg, chatbot], [msg, chatbot])
        submit.click(respond, [msg, chatbot], [msg, chatbot])
        clear.click(lambda: None, None, chatbot, queue=False)
    
    return demo

# Launch chatbot
# demo = create_chatbot_interface(agent)
# demo.launch(share=True)

## 10.2 Multi-Agent Interface

In [ ]:
def create_multi_agent_interface():
    """Create interface with multiple specialized agents"""
    
    with gr.Blocks() as demo:
        gr.Markdown("# Multi-Agent System")
        
        with gr.Tab("General Assistant"):
            chat1 = gr.Chatbot()
            msg1 = gr.Textbox()
            send1 = gr.Button("Send")
        
        with gr.Tab("Customer Support"):
            chat2 = gr.Chatbot()
            msg2 = gr.Textbox()
            send2 = gr.Button("Send")
        
        with gr.Tab("Data Analyst"):
            chat3 = gr.Chatbot()
            msg3 = gr.Textbox()
            send3 = gr.Button("Send")
    
    return demo

# 11. LOGGING & MONITORING

## 11.1 Conversation Logger

In [ ]:
import json
from datetime import datetime

class ConversationLogger:
    """Log conversations for analysis"""
    
    def __init__(self, log_file='conversations.jsonl'):
        self.log_file = log_file
    
    def log_interaction(self, user_message, agent_response, metadata=None):
        """Log a single interaction"""
        log_entry = {
            'timestamp': datetime.now().isoformat(),
            'user_message': user_message,
            'agent_response': agent_response,
            'metadata': metadata or {}
        }
        
        with open(self.log_file, 'a') as f:
            f.write(json.dumps(log_entry) + '\n')
    
    def analyze_logs(self):
        """Analyze conversation logs"""
        logs = []
        with open(self.log_file, 'r') as f:
            for line in f:
                logs.append(json.loads(line))
        
        df = pd.DataFrame(logs)
        
        print("\nConversation Analytics:")
        print(f"Total interactions: {len(df)}")
        print(f"Average response length: {df['agent_response'].str.len().mean():.0f} chars")
        
        return df

logger = ConversationLogger()

## 11.2 Performance Metrics

In [ ]:
import time

def measure_response_time(func, *args):
    """Measure function execution time"""
    start = time.time()
    result = func(*args)
    elapsed = time.time() - start
    return result, elapsed

# Test response time
message = "What's 25 * 17?"
response, latency = measure_response_time(chat, message, agent, memory)
print(f"Response time: {latency:.2f}s")

# 12. DEPLOYMENT

## 12.1 FastAPI REST API

In [ ]:
fastapi_code = '''
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn

app = FastAPI(title="AI Assistant API")

class Message(BaseModel):
    text: str
    user_id: str = "default"

class Response(BaseModel):
    response: str
    timestamp: str

@app.post("/chat", response_model=Response)
async def chat_endpoint(message: Message):
    try:
        response = agent.run(message.text)
        return Response(
            response=response,
            timestamp=datetime.now().isoformat()
        )
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/health")
async def health():
    return {"status": "healthy"}

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''

# 13. CONCLUSIONS & NEXT STEPS

## Summary:
- Agent Type: {CONFIG['agent_type']}
- Tools: {len(tools)}
- Memory: {CONFIG['memory_type']}
- Average Response Time: X.XXs

## Next Steps:
- [ ] Add more specialized tools
- [ ] Implement multi-agent orchestration
- [ ] Add voice input/output
- [ ] Implement user authentication
- [ ] Add conversation analytics dashboard
- [ ] Implement feedback collection
- [ ] Add safety filters and content moderation
- [ ] Deploy to production with scaling
- [ ] Implement A/B testing for prompts